In [1]:
!pip install -q "ray[all]" sklearn

In [2]:
!pip install -qU "ray[tune]" xgboost_ray

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.preprocessing import StandardScaler

In [5]:
from scipy.stats import norm
import numpy as np

import matplotlib as mpl
from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm

In [6]:
from sklearn.metrics import silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax

In [9]:
import hdbscan
from sklearn.cluster import OPTICS

In [10]:
from collections import defaultdict
from collections import Counter

In [11]:
import random

In [12]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [13]:
import shap

In [14]:
from xgboost import plot_importance

from xgboost import to_graphviz

from sklearn.metrics import classification_report,confusion_matrix

In [15]:
import pickle

In [16]:
from typing import Tuple

import ray
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor
from ray.train.xgboost import XGBoostTrainer
from ray.air.config import ScalingConfig
from ray.data.dataset import Dataset
from ray.air.result import Result
from ray.data.preprocessors import StandardScaler

ModuleNotFoundError: No module named 'ray.train'

#### Load data

In [14]:
sampled_workload_data = pd.read_csv('experiments/100_001_sampled_workload_data.csv', index_col=0)
sampled_workload_data

,index,job_name,inst_id,user,task_name,inst_num,status,start_time,end_time,plan_cpu,...,gpu_wrk_util,avg_mem,max_mem,avg_gpu_wrk_mem,max_gpu_wrk_mem,read,write,read_count,write_count,duration
0,9186520,71712e3c961b5bbdb2ce0440,c5be8813bf90aaf9ca6f02f744ccd95596e9850928a498...,2cbd33b08024,ps,5.0,Terminated,4625696.0,4628399.0,600.0,...,7.541667,1.204708,2.723633,0.347158,0.829102,2.068066e+06,3.426616e+05,312.218182,219.376623,2703.0
1,9455559,8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,...,12.049123,1.505760,2.671875,0.445045,0.827148,1.234028e+06,3.724553e+05,264.849650,173.374126,1595.0
2,18718856,b68f25260271d65caccca8e7,5229275f7f92865554f934e97f576ef99db0c079bba59d...,d4d51aca8806,worker,16.0,Terminated,4058070.0,4058321.0,50.0,...,23.727273,1.037900,1.853516,0.445890,0.774414,1.010761e+07,3.781242e+06,2401.913043,590.652174,251.0
3,9501043,8aa3a3a479479aecb41ea324,3808e75bd3204e01e84e33a9f6f873513e52bcc7afb33f...,2cbd33b08024,ps,5.0,Terminated,4642480.0,4644075.0,600.0,...,11.706294,1.481600,2.667969,0.449140,0.829102,2.358233e+06,4.527896e+05,404.843206,271.797909,1595.0
4,11374326,446cd4921e0f00ab5ce1c2ea,12b9ec70cc7c63ebc7184b8a0c363513a8c3cb19746c74...,2cbd33b08024,ps,2.0,Terminated,5521030.0,5526746.0,600.0,...,13.005300,2.357810,3.056641,0.645605,0.827148,1.037695e+06,1.527555e+05,201.449250,181.084731,5716.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,14289977,e49cae3b33b5c205ed8542b0,5283a27d4f19d767256f97c18880a70d69f4d97ca655a3...,47baba33cffe,ps,1.0,Terminated,5890601.0,5893675.0,600.0,...,81.609635,1.940527,1.995117,1.200031,1.213867,1.941028e+06,1.165604e+06,402.442786,171.301824,3074.0
99997,13975013,23f6e20075e637b0976b4ac8,d9a01656ea5d0f9792168159dbb61790d3b0228a633ac8...,47baba33cffe,ps,3.0,Terminated,4297415.0,4306572.0,600.0,...,85.691919,1.803602,2.001953,1.975061,2.213867,1.035467e+06,4.269076e+05,259.100393,115.694335,9157.0
99998,14793315,bf7ee12301073cc75cba61a2,aa5054c3d153f66bf8e1dba6d33b3bb3a9317f1cc7daaa...,dbb3d4806d21,ps,5.0,Terminated,5448779.0,5450118.0,600.0,...,44.271654,1.859159,2.024414,1.172409,1.213867,3.300603e+06,3.495205e+06,568.968627,295.933333,1339.0
99999,14794112,bb95f250974bf5a332f3ea8c,e13b3a5539d62d7b08cad9470ab3b3a33eb54640369337...,dbb3d4806d21,ps,5.0,Terminated,5822706.0,5822846.0,600.0,...,0.000000,0.341797,0.735352,0.000000,0.000000,5.726675e+07,6.596024e+07,8070.600000,3965.400000,140.0


#### Load cluster results

In [15]:
hdbscan_clusterer = pickle.load(open("experiments/hdbscan_300_power_transform_euclidean.pkl", "rb"))

In [16]:
hdbscan_clusterer.labels_

array([ 9,  9,  0, ...,  9, -1, -1])

In [17]:
mask = np.array([1 if x>-1 else 0 for x in hdbscan_clusterer.labels_])
mask

array([1, 1, 1, ..., 1, 0, 0])

## Classification

In [18]:
categories_static_metadata = ['job_name', 'user', 'task_name', 'group', 'workload']

In [19]:
sampled_workload_metadata = sampled_workload_data[categories_static_metadata].astype("category")

In [20]:
sampled_workload_metadata_dummies = pd.get_dummies(sampled_workload_metadata, prefix=categories_static_metadata)

In [21]:
hdbscan_clustered_metadata_cats = sampled_workload_metadata_dummies.values[mask.astype(bool)]

In [22]:
hdbscan_notnoise_labels = hdbscan_clusterer.labels_[mask.astype(bool)]

### XGBoost

In [23]:
X_train, X_test, y_train, y_test = train_test_split(hdbscan_clustered_metadata_cats, hdbscan_notnoise_labels, test_size=0.20, random_state=5)

In [24]:
(X_train.shape, X_test.shape)

((60318, 21547), (15080, 21547))

In [25]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
model = XGBClassifier() 

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
model.get_booster().feature_names = list(sampled_workload_metadata_dummies.columns)

In [ ]:
plot_importance(model, max_num_features=20)

In [ ]:
from xgboost import to_graphviz

to_graphviz(model)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
shap.initjs()

In [18]:
explainer = shap.TreeExplainer(model)

NameError: name 'model' is not defined

In [17]:
shap_values = explainer.shap_values(X_test)

NameError: name 'explainer' is not defined